In [ ]:
BATCH_SIZE = 4
EPOCH_COUNT = 10
VALIDATION_RATIO = 0.2
REGULARIZATION_RATE = 0.0001
LEARNING_RATE = 0.0001

IMAGE_SAMPLE_SIZE = (100, 100, 100, 1)

In [ ]:
DATA_PATH = "/content/drive/My Drive/Medical Research/Data_Tümü/"
PATIENT_LIST_PATH = "/content/drive/My Drive/Medical Research/Hasta_Listesi.xlsx"
LOG_PATH = "/content/drive/My Drive/Medical Research/logs"
MODEL_PATH = "/content/drive/My Drive/Medical Research/models"

In [ ]:
!pip install pydicom 
!pip install pylibjpeg[all]

In [ ]:
!git clone https://github.com/omerenesbayram/lymph-node-metastasis.git

In [ ]:
%cd lymph-node-metastasis/
!git checkout initialization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split as tts
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import metrics

from model import resnet
from data_generator.data_generator import DataGenerator
from data_generator.helper import excel

In [ ]:
pos, neg = excel.get_names(PATIENT_LIST_PATH)

In [ ]:
neg = neg[:len(pos)]

In [ ]:
train_names_pos, val_names_pos = tts(pos, test_size=VALIDATION_RATIO, random_state=42)
train_names_neg, val_names_neg = tts(neg, test_size=VALIDATION_RATIO, random_state=42)

In [ ]:
train_names = np.asarray(train_names_pos + train_names_neg)
np.random.shuffle(train_names)
val_names = np.asarray(val_names_pos + val_names_neg)
np.random.shuffle(val_names)

In [ ]:
print(f'Number of Patients for Training: {len(train_names)}')
print(f'Number of Patients for Validation: {len(val_names)}')

In [ ]:
train_gen = DataGenerator(train_names, "train", PATIENT_LIST_PATH, DATA_PATH, LOG_PATH, batch_size=BATCH_SIZE)
val_gen = DataGenerator(val_names, "train", PATIENT_LIST_PATH, DATA_PATH, LOG_PATH, batch_size=BATCH_SIZE)

In [ ]:
model = resnet.build_resnet_18(input_shape=IMAGE_SAMPLE_SIZE, num_outputs=2, reg_factor=REGULARIZATION_RATE)
metrics = [
    metrics.Accuracy, metrics.Precision, metrics.Recall, 
    metrics.TruePositives, metrics.FalsePositives, metrics.TrueNegatives, metrics.FalseNegatives
]
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=LEARNING_RATE), metrics=metrics)

In [ ]:
from keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir=LOG_PATH)

In [ ]:
model.fit(train_gen, batch_size=BATCH_SIZE, epochs=EPOCH_COUNT, validation_data=val_gen, callbacks = [tensorboard], verbose=2)

In [ ]:
model.save(MODEL_PATH)